# Finding the different regions from maximum projection images of the ventral side of the Paramecium cell
This code processes maximum projection images of the ventral and dorsal sides of expanded Paramecium cells (using ExM) to segment and identify different cellular zones: zone 1 (doublet region), zone 2 (singlet region), and the oral apparatus (OA). Paramecium cells in the images are aligned using "Alignment_Paramecium_images.ipynb" and then a maximum projection image was obtained using ImageJ. The script uses image processing techniques such as thresholding, morphological operations, and contour detection to create binary masks for each zone. The code also smooths these masks and calculates the areas of each region. Finally, it applies the masks to the original images to extract the respective zones, if needed for further analysis.

Input:
- Maximum projection images of the ventral and dorsal sides of Paramecium cells in POLY-E and CENTRIN channels.
- Pixel size for area calculation.
- Parameters for thresholding and morphological operations.

Output:
- Binary masks for zone 1, zone 2, and the oral apparatus (OA).
- Smoothed versions of these masks.
- Extracted images of each zone from the original images.
- Area calculations for each zone based on pixel size.
- Visualizations of intermediate and final results for verification.

In [2]:
import numpy as np
from skimage import io, filters, morphology, measure, draw
from scipy import ndimage
import matplotlib.pyplot as plt
import cv2

### This code is used to separate zone 1 and 2 from max projection images of the ventral side of the cell (and the dorsal side)

In [3]:
# Load the maximum projection image (POLY-E) of the ventral side of the cell. In zone 1, the poly-e channel is more bright. We use this to separate zone 1 and 2.
path = 'W:\\Users\\Daphne\\WT_RESULTS\\WT_ExM_Centrin\\Zone analysis\\'
image_path = path + 'MAX_ExM_ptetWT_Centrin_cell24_polye_rotated_xy_yz_V_zone.tif'
image = io.imread(image_path)
# Convert to 8-bit
image_8bit = (image / np.max(image) * 255).astype(np.uint8)
plt.imshow(image_8bit, cmap='gray')
plt.title('8-bit Image')
plt.show()

# Load the maximum projection image (POLY-E) of the dorsal side of the cell
image_path4 = path + 'MAX_ExM_ptetWT_Centrin_cell24_polye_rotated_xy_yz_D.tif'
image4 = io.imread(image_path4)
# Convert to 8-bit
image_8bitD = (image4 / np.max(image4) * 255).astype(np.uint8)
plt.imshow(image_8bitD, cmap='gray')
plt.title('8-bit Image')
plt.show()


# Load the maximum projection image of the ventral side of the cell CENTRIN
image_path2 = path + 'MAX_ExM_ptetWT_Centrin_cell24_centrin_rotated_xy_yz_V_binary.tif'
image_centrin = io.imread(image_path2)
# Convert to 8-bit
image_8bit_centrin = (image_centrin / np.max(image_centrin) * 255).astype(np.uint8)
plt.imshow(image_8bit_centrin, cmap='gray')
plt.title('8-bit Image')
plt.show()


# Load the maximum projection image of the dorsal side of the cell CENTRIN
image_path3 = path + 'MAX_ExM_ptetWT_Centrin_cell24_centrin_rotated_xy_yz_D_binary.tif'
image_centrinD = io.imread(image_path3)
# Convert to 8-bit
image_8bit_centrinD = (image_centrinD / np.max(image_centrinD) * 255).astype(np.uint8)
plt.imshow(image_8bit_centrinD, cmap='gray')
plt.title('8-bit Image')
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '\\\\sb-nas1.rcp.epfl.ch\\lpl-001\\Users\\Daphne\\WT_RESULTS\\WT_ExM_Centrin\\Zone analysis\\MAX_ExM_ptetWT_Centrin_cell24_polye_rotated_xy_yz_V_zone.tif'

### Use the centrin (or SR) channel to find the cell area for the dorsal and ventral side of the cell


In [ ]:
# Get the total area of the ventral side of the cell
# start with the 8-bit maximum projection image of the ventral side of the cell, which is already loaded as "image_8bit". Threshold it to get the binary mask of the ventral side of the cell
threshold_value4 = filters.threshold_otsu(image_8bit_centrin)
adjusted_threshold_value4 = threshold_value4 * 1  # Adjust the factor as needed
ventral_mask = image_8bit_centrin > adjusted_threshold_value4

# plot the binary mask of the ventral side of the cell
plt.imshow(ventral_mask, cmap='gray')
plt.title('Ventral Mask')
plt.show()

# Get the total area of the dorsal side of the cell
# start with the 8-bit maximum projection image of the ventral side of the cell, which is already loaded as "image_8bit". Threshold it to get the binary mask of the ventral side of the cell
threshold_value5 = filters.threshold_otsu(image_8bit_centrinD)
adjusted_threshold_value5 = threshold_value5 * 1  # Adjust the factor as needed
dorsal_mask = image_8bit_centrinD > adjusted_threshold_value5

# plot the binary mask of the ventral side of the cell
plt.imshow(dorsal_mask, cmap='gray')
plt.title('Dorsal Mask')
plt.show()

In [ ]:
# Apply Gaussian blur
blurred_image = filters.gaussian(image_8bit_centrin, sigma=5)
plt.imshow(blurred_image, cmap='gray')
plt.title('Blurred Image')
plt.show()

# Apply Gaussian blur
blurred_image2 = filters.gaussian(image_8bit_centrinD, sigma=5)
plt.imshow(blurred_image2, cmap='gray')
plt.title('Blurred Image')
plt.show()

In [ ]:
threshold_value4 = filters.threshold_otsu(blurred_image)
adjusted_threshold_value4 = threshold_value4 * 0.7 # Adjust the factor as needed
ventral_mask = blurred_image > adjusted_threshold_value4

# plot the binary mask of the ventral side of the cell
plt.imshow(ventral_mask, cmap='gray')
plt.title('Ventral Mask')
plt.show()


threshold_value5 = filters.threshold_otsu(blurred_image2)
adjusted_threshold_value5 = threshold_value5 * 1  # Adjust the factor as needed
dorsal_mask = blurred_image2 > adjusted_threshold_value5

# plot the binary mask of the ventral side of the cell
plt.imshow(dorsal_mask, cmap='gray')
plt.title('dorsal Mask')
plt.show()

In [ ]:
#### VENTRAL SIDE
# # Draw lines on the image to make (IF NEEDED) connections to create a closed mask
#
# # Ensure the binary mask is in uint8 format with values 0 and 255
# ventral_mask = (ventral_mask * 255).astype(np.uint8)
#
# # Function to draw on the image
# def draw_circle(event, x, y, flags, param):
#     if event == cv2.EVENT_LBUTTONDOWN:
#         cv2.circle(ventral_mask, (x, y), 5, (0), -1)
#
# # Display the image and set the mouse callback function
# cv2.namedWindow('image', cv2.WINDOW_NORMAL)
# cv2.setMouseCallback('image', draw_circle)
#
# # # Resize the window to fit the screen
# # cv2.resizeWindow('image', 400, 300)  # Set the desired window size (width, height)
#
# while True:
#     cv2.imshow('image', ventral_mask)
#     if cv2.waitKey(20) & 0xFF == 27:  # Press 'ESC' to exit
#         break
#
# cv2.destroyAllWindows()
#
# plt.imshow(ventral_mask, cmap='gray')
# plt.title('Mask after adding connections')
# plt.show()

In [ ]:
# DORSAL SIDE
# # Draw lines on the image to make (IF NEEDED) connections to create a closed mask
#
# # Ensure the binary mask is in uint8 format with values 0 and 255
# dorsal_mask = (dorsal_mask * 255).astype(np.uint8)
#
# # Function to draw on the image
# def draw_circle(event, x, y, flags, param):
#     if event == cv2.EVENT_LBUTTONDOWN:
#         cv2.circle(dorsal_mask, (x, y), 5, (0), -1)
#
# # Display the image and set the mouse callback function
# cv2.namedWindow('image', cv2.WINDOW_NORMAL)
# cv2.setMouseCallback('image', draw_circle)
#
# # # Resize the window to fit the screen
# # cv2.resizeWindow('image', 400, 300)  # Set the desired window size (width, height)
#
# while True:
#     cv2.imshow('image', dorsal_mask)
#     if cv2.waitKey(20) & 0xFF == 27:  # Press 'ESC' to exit
#         break
#
# cv2.destroyAllWindows()
#
# plt.imshow(dorsal_mask, cmap='gray')
# plt.title('Mask after adding connections')
# plt.show()

In [ ]:
# VENTRAL SIDE
# Fill holes in the binary mask
filled_mask = ndimage.binary_fill_holes(ventral_mask)
plt.imshow(filled_mask, cmap='gray')
plt.title('Filled Mask')
plt.show()

# remove the cilia from the cell surface by applying erosion and later dilation
eroded_mask = filled_mask.copy()
iterations = 75  # Set the number of iterations as needed to remove the cilia

for _ in range(iterations):
    eroded_mask = morphology.erosion(eroded_mask, morphology.disk(1))

plt.imshow(eroded_mask, cmap='gray')
plt.title('Eroded Mask')
plt.show()

# Apply dilation
dilated_mask = eroded_mask.copy()

for _ in range(iterations):
    dilated_mask = morphology.dilation(dilated_mask, morphology.disk(1))

plt.imshow(dilated_mask, cmap='gray')
plt.title('Dilated Mask')
plt.show()

In [ ]:
# DORSAL SIDE
# Fill holes in the binary mask
filled_mask2 = ndimage.binary_fill_holes(dorsal_mask)
plt.imshow(filled_mask2, cmap='gray')
plt.title('Filled Mask2')
plt.show()

# remove the cilia from the cell surface by applying erosion and later dilation
eroded_mask4 = filled_mask2.copy()
iterations = 60  # Set the number of iterations as needed to remove the cilia

for _ in range(iterations):
    eroded_mask4 = morphology.erosion(eroded_mask4, morphology.disk(1))

plt.imshow(eroded_mask4, cmap='gray')
plt.title('Eroded Mask')
plt.show()

# Apply dilation
dilated_mask4 = eroded_mask4.copy()

for _ in range(iterations):
    dilated_mask4 = morphology.dilation(dilated_mask4, morphology.disk(1))

plt.imshow(dilated_mask4, cmap='gray')
plt.title('Dilated Mask')
plt.show()

In [ ]:
# Save the ventral mask
output_path = image_path2[:-4] + '_ventral_mask.tif'
ventral_mask = dilated_mask.astype(np.uint8) * 255
io.imsave(output_path, ventral_mask)

# Save the dorsal mask
output_path = image_path3[:-4] + '_dorsal_mask.tif'
dorsal_mask = dilated_mask4.astype(np.uint8) * 255
io.imsave(output_path, dorsal_mask)

#### Find zone 1 + OA using the poly-e channel, ventral side

In [ ]:
# Apply auto threshold and adjust it to be lower or higher
threshold_value = filters.threshold_otsu(image_8bit)
# adjusted_threshold_value = threshold_value * 0.7  # Adjust the factor as needed
adjusted_threshold_value = threshold_value * 0.3  # Adjust the factor as needed
binary_mask = image_8bit > adjusted_threshold_value

# Plot to check if it is correctly thresholded
plt.imshow(binary_mask, cmap='gray')
plt.title('Binary Mask with Threshold')
plt.show()

In [ ]:
# Convert to mask
binary_mask = binary_mask.astype(np.uint8)
plt.imshow(binary_mask, cmap='gray')
plt.title('Binary Mask')
plt.show()

# Apply Gaussian blur, to connect close-by BBs
blurred_image = filters.gaussian(binary_mask, sigma=2)
plt.imshow(blurred_image, cmap='gray')
plt.title('Blurred Image')
plt.show()

# Convert to mask again
binary_mask = blurred_image > filters.threshold_otsu(blurred_image)
plt.imshow(binary_mask, cmap='gray')
plt.title('Binary Mask after Blurring')
plt.show()

# Fill holes
filled_mask = ndimage.binary_fill_holes(binary_mask)
plt.imshow(filled_mask, cmap='gray')
plt.title('Filled Mask')
plt.show()

# Shape filter (filter by area)
label_image = measure.label(filled_mask)
properties = measure.regionprops(label_image)
filtered_mask = np.zeros_like(filled_mask)

for prop in properties:
    if prop.area >= 20000:
        filtered_mask[label_image == prop.label] = 1

plt.imshow(filtered_mask, cmap='gray')
plt.title('Filtered Mask')
plt.show()

In [ ]:
# Draw lines on the image to remove unwanted parts from the main body (if necessary)

# Ensure the binary mask is in uint8 format with values 0 and 255
filtered_mask = (filtered_mask * 255).astype(np.uint8)

# Function to draw on the image
def draw_circle(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(filtered_mask, (x, y), 5, (0), -1)

# Display the image and set the mouse callback function
cv2.namedWindow('image')
cv2.setMouseCallback('image', draw_circle)

while True:
    cv2.imshow('image', filtered_mask)
    if cv2.waitKey(20) & 0xFF == 27:  # Press 'ESC' to exit
        break

cv2.destroyAllWindows()

plt.imshow(filtered_mask, cmap='gray')
plt.title('Filtered Mask after removing connections to unwanted parts')
plt.show()

In [ ]:
# Shape filter again (filter by area) to remove the loose ends
label_image = measure.label(filtered_mask)
properties = measure.regionprops(label_image)
filtered_mask = np.zeros_like(filtered_mask)

for prop in properties:
    if prop.area >= 120000:
        filtered_mask[label_image == prop.label] = 1

plt.imshow(filtered_mask, cmap='gray')
plt.title('Filtered Mask after Second Filtering')
plt.show()

In [ ]:
# Apply dilation
iterations = 50  # Set the number of iterations as needed, so that there are no holes in the mask
dilated_mask = filtered_mask.copy()

for _ in range(iterations):
    dilated_mask = morphology.dilation(dilated_mask, morphology.disk(1))

plt.imshow(dilated_mask, cmap='gray')
plt.title('Dilated Mask')
plt.show()

# Apply erosion, the same number of iterations are used as for the dilation
eroded_mask = dilated_mask.copy()

for _ in range(iterations):
    eroded_mask = morphology.erosion(eroded_mask, morphology.disk(1))

plt.imshow(eroded_mask, cmap='gray')
plt.title('Eroded Mask')
plt.show()

In [ ]:
# Save the final result
output_path = image_path[:-4] + '_mask_zone1+OA.tif'
io.imsave(output_path, eroded_mask.astype(np.uint8) * 255)

zone1_OA_mask = eroded_mask

In [ ]:
# Smooth the mask of zone 1 and OA and save it

# Apply Gaussian blur to the eroded mask to smooth it
blurred_eroded_mask = filters.gaussian(eroded_mask, sigma=20)

# Threshold the blurred image to create a binary mask
smoothed_mask = blurred_eroded_mask > filters.threshold_otsu(blurred_eroded_mask)

# Find contours of the smoothed binary mask
contours = measure.find_contours(smoothed_mask, level=0.5)

# Fit a polygon around the shape
polygon = measure.approximate_polygon(contours[0], tolerance=3)

# Create an empty mask to draw the polygon
polygon_mask = np.zeros_like(smoothed_mask)

# Draw the polygon on the mask
rr, cc = draw.polygon(polygon[:, 0], polygon[:, 1], polygon_mask.shape)
polygon_mask[rr, cc] = 1

# Display the blurred mask
plt.imshow(blurred_eroded_mask, cmap='gray')
plt.title('Blurred Mask')
plt.show()

# Display the resulting polygon mask
plt.imshow(polygon_mask, cmap='gray')
plt.title('Polygon Mask')
plt.show()

# Ensure the smoothed mask is binary
eroded_mask = (eroded_mask > 0).astype(np.uint8)
polygon_mask_zone1_OA = (polygon_mask > 0).astype(np.uint8)

# Load the pixel size of the image
pixel_size = 0.287  # Set the pixel size in micrometers

# Calculate the area of the smoothed mask
eroded_area = np.sum(eroded_mask) * pixel_size ** 2
smoothed_area = np.sum(polygon_mask_zone1_OA) * pixel_size ** 2

# Print the area
print(f'Eroded Mask Area: {eroded_area} µm^2')
print(f'Smoothed Mask Area: {smoothed_area} µm^2')

# Save the smoothed zone 1
output_path = image_path[:-4] + '_mask_zone1+OA_smoothed.tif'
io.imsave(output_path, polygon_mask_zone1_OA.astype(np.uint8) * 255)

#### The next part of the code extracts the oral apparatus (OA) from the ventral side of the Paramecium cell to extract later from zone 1


In [ ]:
# Start again from the 8-bit maximum projection image of the ventral side of the cell, which is already loaded as "image_8bit"
#show the image
plt.imshow(image_8bit, cmap='gray')
plt.title('8-bit Image')
plt.show()

# Apply auto threshold and adjust it to be lower or higher --> for this part we want only the very bright signal
threshold_value2 = filters.threshold_otsu(image_8bit)
adjusted_threshold_value2 = threshold_value2 * 2  # Adjust the factor as needed
binary_mask2 = image_8bit > adjusted_threshold_value2

# Plot to check if it is correctly thresholded
plt.imshow(binary_mask2, cmap='gray')
plt.title('Binary Mask with Threshold')
plt.show()

In [ ]:
# Apply erosion
eroded_mask2 = binary_mask2.copy()
iterations2 = 2  # Set the number of iterations as needed to remove (connections to and between) the BBs outside the OA

for _ in range(iterations2):
    eroded_mask2 = morphology.erosion(eroded_mask2, morphology.disk(1))

plt.imshow(eroded_mask2, cmap='gray')
plt.title('Eroded Mask')
plt.show()


# Shape filter (filter by area)
label_image2 = measure.label(eroded_mask2)
properties2 = measure.regionprops(label_image2)
filtered_mask2 = np.zeros_like(eroded_mask2)

for prop in properties2:
    if prop.area >= 2000:
        filtered_mask2[label_image2 == prop.label] = 1

plt.imshow(filtered_mask2, cmap='gray')
plt.title('Filtered Mask')
plt.show()


# Apply dilation
dilated_mask2 = filtered_mask2.copy()
iterations3 = 60  # Set the number of iterations as needed to fill the holes in the OA

for _ in range(iterations3):
    dilated_mask2 = morphology.dilation(dilated_mask2, morphology.disk(1))

plt.imshow(dilated_mask2, cmap='gray')
plt.title('Dilated Mask')
plt.show()


# Apply erosion, the same number of iterations are used as for the dilation
eroded_mask3 = dilated_mask2.copy()

for _ in range(iterations3):
# for _ in range(1):
    eroded_mask3 = morphology.erosion(eroded_mask3, morphology.disk(1))

plt.imshow(eroded_mask3, cmap='gray')
plt.title('Eroded Mask')
plt.show()

In [ ]:
# ### IF NEEDED, draw lines on the image to remove unwanted parts from the main body of the OA and filter
# # Draw lines on the image to remove the connections with the thin lines around the OA (if necessary)
#
# # Ensure the binary mask is in uint8 format with values 0 and 255
# eroded_mask3 = (eroded_mask3 * 255).astype(np.uint8)
#
# # Function to draw on the image
# def draw_circle(event, x, y, flags, param):
#     if event == cv2.EVENT_LBUTTONDOWN:
#         cv2.circle(eroded_mask3, (x, y), 5, (0), -1)
#
# # Display the image and set the mouse callback function
# cv2.namedWindow('image')
# cv2.setMouseCallback('image', draw_circle)
#
# while True:
#     cv2.imshow('image', eroded_mask3)
#     if cv2.waitKey(20) & 0xFF == 27:  # Press 'ESC' to exit
#         break
#
# cv2.destroyAllWindows()
#
# plt.imshow(eroded_mask3, cmap='gray')
# plt.title('Mask after removing connections to unwanted parts')
# plt.show()
#
# # Shape filter (filter by area) to remove the loose ends
# label_image = measure.label(eroded_mask3)
# properties = measure.regionprops(label_image)
# eroded_mask3 = np.zeros_like(eroded_mask3)
#
# for prop in properties:
#     if prop.area >= 50000:
#         eroded_mask3[label_image == prop.label] = 1
#
# plt.imshow(eroded_mask3, cmap='gray')
# plt.title('OA after removing extra parts')
# plt.show()

In [ ]:
# Save the OA mask
output_path = image_path[:-4] + '_OA_mask.tif'
OA_mask = eroded_mask3
io.imsave(output_path, eroded_mask3.astype(np.uint8) * 255)

In [ ]:
# Smooth the mask of the OA and save it

# Apply Gaussian blur to the eroded mask to smooth it
blurred_mask2 = filters.gaussian(eroded_mask3, sigma=20)

# Threshold the blurred image to create a binary mask
smoothed_mask2 = blurred_mask2 > filters.threshold_otsu(blurred_mask2)

# Find contours of the smoothed binary mask
contours2 = measure.find_contours(smoothed_mask2, level=0.5)

# Fit a polygon around the shape
polygon2 = measure.approximate_polygon(contours2[0], tolerance=3)

# Create an empty mask to draw the polygon
polygon_mask2 = np.zeros_like(smoothed_mask2)

# Draw the polygon on the mask
rr, cc = draw.polygon(polygon2[:, 0], polygon2[:, 1], polygon_mask2.shape)
polygon_mask2[rr, cc] = 1

# Display the blurred mask
plt.imshow(blurred_mask2, cmap='gray')
plt.title('Blurred Mask')
plt.show()

# Display the smoothed mask
plt.imshow(smoothed_mask2, cmap='gray')
plt.title('Smoothed Mask')
plt.show()

# Display the resulting polygon mask
plt.imshow(polygon_mask2, cmap='gray')
plt.title('Polygon Mask')
plt.show()

# Ensure the smoothed mask is binary
eroded_mask3 = (eroded_mask3 > 0).astype(np.uint8)
polygon_mask_OA = (polygon_mask2 > 0).astype(np.uint8)

# Load the pixel size of the image
pixel_size = 0.287  # Set the pixel size in micrometers

# Calculate the area of the smoothed mask
OA_area = np.sum(eroded_mask3) * pixel_size ** 2
smoothed_area_OA = np.sum(polygon_mask_OA) * pixel_size ** 2

# Print the area
print(f'Eroded Mask Area: {OA_area} µm^2')
print(f'Smoothed Mask Area: {smoothed_area_OA} µm^2')

# Save the smoothed OA
output_path = image_path[:-4] + '_OA_mask_smoothed.tif'
io.imsave(output_path, polygon_mask_OA.astype(np.uint8) * 255)

In [ ]:
# Subtract eroded_mask3 from eroded_mask to remove the OA from zone 1
# ensure the binary masks are in uint8 format with values 0 and 1
OA_mask = (OA_mask > 0).astype(np.uint8)
zone1_OA_mask = (zone1_OA_mask > 0).astype(np.uint8)

# invert the OA mask and zone 1 mask
inverted_OA_mask = np.logical_not(OA_mask).astype(np.uint8)
inverted_zone1_OA_mask = np.logical_not(zone1_OA_mask).astype(np.uint8)

# zone1_mask = eroded_mask - eroded_mask3
zone1_mask = zone1_OA_mask - OA_mask

plt.imshow(OA_mask, cmap='gray')
plt.title('Zone 1 Mask')
plt.show()
plt.imshow(zone1_OA_mask, cmap='gray')
plt.title('Zone 1 Mask')
plt.show()
plt.imshow(zone1_mask, cmap='gray')
plt.title('Zone 1 Mask')
plt.show()

In [ ]:
# Draw lines on the image to remove the connections with the thin lines around the OA (if necessary)

# Ensure the binary mask is in uint8 format with values 0 and 255
zone1_mask = (zone1_mask * 255).astype(np.uint8)

# Function to draw on the image
def draw_circle(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(zone1_mask, (x, y), 5, (0), -1)

# Display the image and set the mouse callback function
cv2.namedWindow('image')
cv2.setMouseCallback('image', draw_circle)

while True:
    cv2.imshow('image', zone1_mask)
    if cv2.waitKey(20) & 0xFF == 27:  # Press 'ESC' to exit
        break

cv2.destroyAllWindows()

plt.imshow(zone1_mask, cmap='gray')
plt.title('Mask after removing connections to unwanted parts')
plt.show()

# Shape filter (filter by area) to remove the loose ends
label_image = measure.label(zone1_mask)
properties = measure.regionprops(label_image)
zone1_mask = np.zeros_like(zone1_mask)

for prop in properties:
    if prop.area >= 80000:
        zone1_mask[label_image == prop.label] = 1

plt.imshow(zone1_mask, cmap='gray')
plt.title('Zone 1 after removing the OA')
plt.show()

In [ ]:
# Save the mask of zone 1
output_path = image_path[:-4] + '_mask_zone1.tif'
io.imsave(output_path, zone1_mask.astype(np.uint8) * 255)

In [ ]:
# Smooth the mask of zone 1 and OA and save it

# Apply Gaussian blur to the eroded mask to smooth it
blurred_zone1_mask = filters.gaussian(zone1_mask, sigma=20)

# Threshold the blurred image to create a binary mask
smoothed_mask = blurred_zone1_mask > filters.threshold_otsu(blurred_zone1_mask)

# Find contours of the smoothed binary mask
contours = measure.find_contours(smoothed_mask, level=0.5)

# Fit a polygon around the shape
polygon = measure.approximate_polygon(contours[0], tolerance=3)

# Create an empty mask to draw the polygon
polygon_mask = np.zeros_like(smoothed_mask)

# Draw the polygon on the mask
rr, cc = draw.polygon(polygon[:, 0], polygon[:, 1], polygon_mask.shape)
polygon_mask[rr, cc] = 1

# Display the blurred mask
plt.imshow(blurred_zone1_mask, cmap='gray')
plt.title('Blurred Mask')
plt.show()

# Display the smoothed mask
plt.imshow(smoothed_mask, cmap='gray')
plt.title('Smoothed Mask')
plt.show()

# Display the resulting polygon mask
plt.imshow(polygon_mask, cmap='gray')
plt.title('Polygon Mask')
plt.show()

# Ensure the smoothed mask is binary
zone1_mask = (zone1_mask > 0).astype(np.uint8)
polygon_mask_zone1 = (polygon_mask > 0).astype(np.uint8)

# Load the pixel size of the image
pixel_size = 0.287  # Set the pixel size in micrometers

# Calculate the area of the smoothed mask
zone1_area = np.sum(zone1_mask) * pixel_size ** 2
blurred_area = np.sum(blurred_zone1_mask) * pixel_size ** 2
smoothed_area_zone1 = np.sum(polygon_mask_zone1) * pixel_size ** 2

# Print the area
print(f'Eroded Mask Area: {zone1_area} µm^2')
print(f'Blurred Mask Area: {blurred_area} µm^2')
print(f'Smoothed Mask Area: {smoothed_area_zone1} µm^2')

# Save the smoothed zone 1
output_path = image_path[:-4] + '_mask_zone1_smoothed.tif'
io.imsave(output_path, polygon_mask_zone1.astype(np.uint8) * 255)

In [ ]:
# IF THE DORSAL SIDE OF THE IMAGE HAS A PART OF ZONE 1
##########################
############################### NOT NEEDED SO FAR



In [ ]:
# Mask of zone 1 (ventral side)
# Overlap zone 1 with the mask of the ventral side of the cell to get the mask of zone 1
zone1_mask = zone1_mask * ventral_mask
polygon_mask_zone1 = polygon_mask_zone1 * ventral_mask

# Save
output_path = image_path[:-4] + '_mask_zone1.tif'
io.imsave(output_path, zone1_mask.astype(np.uint8) * 255)
output_path = image_path[:-4] + '_mask_zone1_smoothed.tif'
io.imsave(output_path, polygon_mask_zone1.astype(np.uint8) * 255)

# OA overlap with the mask of the ventral side of the cell to get the mask of OA
OA_mask = OA_mask * ventral_mask
polygon_mask_OA = polygon_mask_OA * ventral_mask

# Save
output_path = image_path[:-4] + '_mask_OA.tif'
io.imsave(output_path, OA_mask.astype(np.uint8) * 255)
output_path = image_path[:-4] + '_mask_OA_smoothed.tif'
io.imsave(output_path, polygon_mask_OA.astype(np.uint8) * 255)

# Show the masks
plt.imshow(zone1_mask, cmap='gray')
plt.title('Mask of Zone 1')
plt.show()
plt.imshow(polygon_mask_zone1, cmap='gray')
plt.title('Mask of Zone 1 smoothed')
plt.show()
plt.imshow(OA_mask, cmap='gray')
plt.title('Mask of OA')
plt.show()
plt.imshow(polygon_mask_OA, cmap='gray')
plt.title('Mask of OA smoothed')
plt.show()

In [ ]:
# Mask of zone 2 (ventral side)
# Subtract the mask of zone 1 + OA from the mask of the ventral side of the cell to get the mask of zone 2
zone2_mask = (ventral_mask - (255- (zone1_mask + OA_mask)))
zone2_mask_smoothed = (ventral_mask - (255 -(polygon_mask_zone1 + polygon_mask_OA)))

# Ensure the binary masks are in uint8 format with values 0 and 1
zone2_mask = (zone2_mask > 0).astype(np.uint8)
zone2_mask_smoothed = (zone2_mask_smoothed > 0).astype(np.uint8)

# invert the zone 2 masks
zone2_mask = np.logical_not(zone2_mask).astype(np.uint8)
zone2_mask_smoothed = np.logical_not(zone2_mask_smoothed).astype(np.uint8)

# Save the mask of zone 2
output_path = image_path2[:-4] + '_zone2_mask.tif'
io.imsave(output_path, zone2_mask * 255)

output_path = image_path2[:-4] + '_zone2_mask_smoothed.tif'
io.imsave(output_path, zone2_mask_smoothed * 255)

# show the mask of zone 2
plt.imshow(zone2_mask, cmap='gray')
plt.title('Mask of Zone 2')
plt.show()

plt.imshow(zone2_mask_smoothed, cmap='gray')
plt.title('Mask of Zone 2 smoothed')
plt.show()

#### Now that we have the mask of zone 1, zone 2 and the masks of the ventral and dorsal side of the cell, we can use it to extract the zones from the original images.


In [ ]:
# Sort all images
# inverted poly-e channels ventral and dorsal side
polye_ventral_image = 255- image_8bit
polye_dorsal_image = 255- image_8bitD

# centrin channels ventral and dorsal side
centrin_ventral_image = 255- image_8bit_centrin
centrin_dorsal_image = 255- image_8bit_centrinD

# Apply the masks of the dorsal and ventral side to the images to get rid of signal outside the cell
polye_ventral_image = polye_ventral_image * ventral_mask
polye_dorsal_image = polye_dorsal_image * dorsal_mask
centrin_ventral_image = centrin_ventral_image * ventral_mask
centrin_dorsal_image = centrin_dorsal_image * dorsal_mask

# Show the images
plt.imshow(polye_ventral_image, cmap='gray')
plt.title('Poly-e Ventral Image')
plt.show()
plt.imshow(polye_dorsal_image, cmap='gray')
plt.title('Poly-e Dorsal Image')
plt.show()
plt.imshow(centrin_ventral_image, cmap='gray')
plt.title('Centrin Ventral Image')
plt.show()
plt.imshow(centrin_dorsal_image, cmap='gray')
plt.title('Centrin Dorsal Image')
plt.show()

In [ ]:
# Apply the masks to the images to extract the zones
# # ZONE 1
# zone1_ventral_image_polye = zone1_mask * polye_ventral_image
# zone1_ventral_image_centrin = zone1_mask * centrin_ventral_image
# ZONE 1 smoothed
zone1_ventral_image_polye = polygon_mask_zone1 * (255- polye_ventral_image)
zone1_ventral_image_centrin = polygon_mask_zone1 * (255- centrin_ventral_image)

# # OA
# OA_ventral_image_polye = OA_mask * (255 - polye_ventral_image)
# OA_ventral_image_centrin = OA_mask * (255 - centrin_ventral_image)
# OA smoothed
OA_ventral_image_polye = polygon_mask_OA * (255 - polye_ventral_image)
OA_ventral_image_centrin = polygon_mask_OA * (255 - centrin_ventral_image)

# # ZONE 2 ventral
# zone2_ventral_image_polye = zone2_mask * (255 - polye_ventral_image)
# zone2_ventral_image_centrin = zone2_mask * (255 - centrin_ventral_image)
# ZONE 2 ventral smoothed
zone2_ventral_image_polye = zone2_mask_smoothed * polye_ventral_image
zone2_ventral_image_centrin = zone2_mask_smoothed * centrin_ventral_image
# ZONE 2 dorsal
zone2_dorsal_image_polye = dorsal_mask * (255 - polye_dorsal_image)
zone2_dorsal_image_centrin = dorsal_mask * (255 - centrin_dorsal_image)

# show the images
plt.imshow(zone1_ventral_image_polye, cmap='gray')
plt.title('Zone 1 Poly-e')
plt.show()
plt.imshow(zone1_ventral_image_centrin, cmap='gray')
plt.title('Zone 1 Centrin')
plt.show()
plt.imshow(OA_ventral_image_polye, cmap='gray')
plt.title('OA Poly-e')
plt.show()
plt.imshow(OA_ventral_image_centrin, cmap='gray')
plt.title('OA Centrin')
plt.show()
plt.imshow(zone2_ventral_image_polye, cmap='gray')
plt.title('Zone 2 Poly-e ventral')
plt.show()
plt.imshow(zone2_ventral_image_centrin, cmap='gray')
plt.title('Zone 2 Centrin ventral')
plt.show()
plt.imshow(zone2_dorsal_image_polye, cmap='gray')
plt.title('Zone 2 Poly-e dorsal')
plt.show()
plt.imshow(zone2_dorsal_image_centrin, cmap='gray')
plt.title('Zone 2 Centrin dorsal')
plt.show()

# Save the images
output_path = image_path[:-4] + '_zone1_ventral_polye.tif'
io.imsave(output_path, zone1_ventral_image_polye.astype(np.uint8))

output_path = image_path2[:-4] + '_zone1_ventral_centrin.tif'
io.imsave(output_path, zone1_ventral_image_centrin.astype(np.uint8))

output_path = image_path[:-4] + '_OA_ventral_polye.tif'
io.imsave(output_path, OA_ventral_image_polye.astype(np.uint8))

output_path = image_path2[:-4] + '_OA_ventral_centrin.tif'
io.imsave(output_path, OA_ventral_image_centrin.astype(np.uint8))

output_path = image_path[:-4] + '_zone2_ventral_polye.tif'
io.imsave(output_path, zone2_ventral_image_polye.astype(np.uint8))

output_path = image_path2[:-4] + '_zone2_ventral_centrin.tif'
io.imsave(output_path, zone2_ventral_image_centrin.astype(np.uint8))

output_path = image_path4[:-4] + '_zone2_dorsal_polye.tif'
io.imsave(output_path, zone2_dorsal_image_polye.astype(np.uint8))

output_path = image_path3[:-4] + '_zone2_dorsal_centrin.tif'
io.imsave(output_path, zone2_dorsal_image_centrin.astype(np.uint8))

#### Calculate the areas of the zones

In [ ]:
# Measure the area of the ventral side of the cell, the OA, and zone 1 and 2
# Ensure the masks are binary
ventral_mask = (ventral_mask > 0).astype(np.uint8)
OA_mask = (OA_mask > 0).astype(np.uint8)
zone1_mask = (zone1_mask > 0).astype(np.uint8)
zone2_mask = (zone2_mask > 0).astype(np.uint8)
dorsal_mask = (dorsal_mask > 0).astype(np.uint8)
polygon_mask_zone1 = (polygon_mask_zone1 > 0).astype(np.uint8)
polygon_mask_OA = (polygon_mask_OA > 0).astype(np.uint8)
zone2_mask_smoothed = (zone2_mask_smoothed > 0).astype(np.uint8)

# Load the pixel size of the image
pixel_size = 0.287  # Set the pixel size in micrometers

# Calculate the area of the ventral & dorsal side of the cell
ventral_area = np.sum(ventral_mask) * pixel_size ** 2
dorsal_area = np.sum(dorsal_mask) * pixel_size ** 2


# Calculate the area of the OA
OA_area = np.sum(OA_mask) * pixel_size ** 2
# Calculate the area of zone 1
zone1_area = np.sum(zone1_mask) * pixel_size ** 2
# Calculate the area of zone 2 on the ventral side of the cell
zone2_area_ventral = np.sum(zone2_mask) * pixel_size ** 2
# Calculate the total area of zone 2 of the cell = zone2_area_ventral + dorsal_area
zone2_area_total = zone2_area_ventral + dorsal_area


# Calculate the sum of the areas of OA, zone 1, and zone 2. This is to check that the sum of the areas of the masks is equal to the area of the ventral side of the cell!
total_area_ventral = OA_area + zone1_area + zone2_area_ventral
total_area2 = total_area_ventral + dorsal_area
total_area = ventral_area + dorsal_area

print(f'Ventral Area: {ventral_area} µm^2')
print(f'Dorsal Area: {dorsal_area} µm^2')
print(f'total area cell (dorsal + ventral): {total_area} µm^2')
print(f'OA Area: {OA_area} µm^2')
print(f'Zone 1 Area: {zone1_area} µm^2')
print(f'Zone 2 Area ventral: {zone2_area_ventral} µm^2')
print(f'Zone 2 Area total: {zone2_area_total} µm^2')
print(f'Total Area of ventral + dorsal side cell: {total_area} µm^2')
print(f'Total Area of OA, Zone 1, and Zone 2 (ventral + dorsal): {total_area2} µm^2')
print(f'difference between total area of cell and sum of zone areas: {total_area - total_area2} µm^2')


# Check for overlaps between the masks (ventral side), they should not overlap
overlap_OA_zone1 = np.sum(OA_mask & zone1_mask)
overlap_OA_zone2 = np.sum(OA_mask & zone2_mask)
overlap_zone1_zone2 = np.sum(zone1_mask & zone2_mask)
print(f'Overlap between OA and Zone 1 on ventral side cell: {overlap_OA_zone1} pixels')
print(f'Overlap between OA and Zone 2 on ventral side cell: {overlap_OA_zone2} pixels')
print(f'Overlap between Zone 1 and Zone 2 on ventral side cell: {overlap_zone1_zone2} pixels')


# Calculate the percentages
OA_percentage = (OA_area / total_area) * 100
zone1_percentage = (zone1_area / total_area) * 100
zone2_percentage = (zone2_area_total / total_area) * 100
total_percentage = OA_percentage + zone1_percentage + zone2_percentage
print(f'Percentage of the cell occupied by the OA: {OA_percentage:.2f}%')
print(f'Percentage of the cell occupied by Zone 1: {zone1_percentage:.2f}%')
print(f'Percentage of the cell occupied by Zone 2: {zone2_percentage:.2f}%')
print(f'Total Percentage: {total_percentage:.2f}%')


# Smoothed masks
# Calculate the area of the smoothed masks
smoothed_area_OA = np.sum(polygon_mask_OA) * pixel_size ** 2
smoothed_area_zone1 = np.sum(polygon_mask_zone1) * pixel_size ** 2
smoothed_area_zone2 = np.sum(zone2_mask_smoothed) * pixel_size ** 2
print(f'Smoothed OA Area: {smoothed_area_OA} µm^2')
print(f'Smoothed Zone 1 Area: {smoothed_area_zone1} µm^2')
print(f'Smoothed Zone 2 Area: {smoothed_area_zone2} µm^2')
# total ventral area the same as the smoothed areas combined?
total_area_smoothed = smoothed_area_OA + smoothed_area_zone1 + smoothed_area_zone2
print(f'Total Area of OA, Zone 1, and Zone 2 (ventral): {total_area_smoothed} µm^2')
print(f'difference between total area of cell and sum of smoothed zone areas (all ventral side): {total_area_ventral - total_area_smoothed} µm^2')
smoothed_OA_percentage = (smoothed_area_OA / total_area) * 100
smoothed_zone1_percentage = (smoothed_area_zone1 / total_area) * 100
smoothed_zone2_percentage = ((smoothed_area_zone2 + dorsal_area) / total_area) * 100
total_smoothed_percentage = smoothed_OA_percentage + smoothed_zone1_percentage + smoothed_zone2_percentage
print(f'Percentage of the cell occupied by the OA: {smoothed_OA_percentage:.2f}%')
print(f'Percentage of the cell occupied by Zone 1: {smoothed_zone1_percentage:.2f}%')
print(f'Percentage of the cell occupied by Zone 2: {smoothed_zone2_percentage:.2f}%')
print(f'Total Percentage: {total_smoothed_percentage:.2f}%')

# Define the file path for the summary text file
summary_file_path = image_path[:-4] + 'summary_cell_zones.txt'

# Open the file in write mode
with open(summary_file_path, 'w') as file:
    # Write the summary results to the file
    file.write(f'Image Path: {image_path}\n')
    file.write(f'Pixel Size: {pixel_size} µm\n')
    file.write(f'Ventral Area: {ventral_area} µm^2\n')
    file.write(f'Dorsal Area: {dorsal_area} µm^2\n')
    file.write(f'OA Area: {OA_area} µm^2\n')
    file.write(f'Zone 1 Area: {zone1_area} µm^2\n')
    file.write(f'Zone 2 Area ventral: {zone2_area_ventral} µm^2\n')
    file.write(f'Zone 2 Area total: {zone2_area_total} µm^2\n')
    file.write(f'Total Area of ventral + dorsal side cell: {total_area} µm^2\n')
    file.write(f'Total Area of OA, Zone 1, and Zone 2 (ventral + dorsal): {total_area2} µm^2\n')
    file.write(f'difference between total area of cell and sum of zone areas: {total_area - total_area2} µm^2\n')
    file.write(f'Overlap between OA and Zone 1 on ventral side cell: {overlap_OA_zone1} pixels\n')
    file.write(f'Overlap between OA and Zone 2 on ventral side cell: {overlap_OA_zone2} pixels\n')
    file.write(f'Overlap between Zone 1 and Zone 2 on ventral side cell: {overlap_zone1_zone2} pixels\n')
    file.write(f'Percentage of the cell occupied by the OA: {OA_percentage:.2f}%\n')
    file.write(f'Percentage of the cell occupied by Zone 1: {zone1_percentage:.2f}%\n')
    file.write(f'Percentage of the cell occupied by Zone 2: {zone2_percentage:.2f}%\n')
    file.write(f'Total Percentage: {total_percentage:.2f}%\n')
    file.write(f'Smoothed OA Area: {smoothed_area_OA} µm^2\n')
    file.write(f'Smoothed Zone 1 Area: {smoothed_area_zone1} µm^2\n')
    file.write(f'Smoothed Zone 2 Area: {smoothed_area_zone2} µm^2\n')
    file.write(f'Total Area of OA, Zone 1, and Zone 2 (ventral): {total_area_smoothed} µm^2\n')
    file.write(f'difference between total area of cell and sum of smoothed zone areas (all ventral side): {total_area - total_area_smoothed} µm^2\n')
    file.write(f'Percentage of the cell occupied by the smoothed OA: {smoothed_OA_percentage:.2f}%\n')
    file.write(f'Percentage of the cell occupied by smoothed Zone 1: {smoothed_zone1_percentage:.2f}%\n')
    file.write(f'Percentage of the cell occupied by smoothed Zone 2: {smoothed_zone2_percentage:.2f}%\n')
    file.write(f'Total Percentage of smoothed zones&OA: {total_smoothed_percentage:.2f}%\n')
    file.write(f'a percentage slightly higher than 100% is due to the smoothed overlap between zone 1 and the OA\n')



### Now we have the areas and percentages of the different cell zones
### For analysis, remove the outer edge of the cell to avoid the max projection effects on the edges which are oriented differently
##### We need the erosion on the mask to apply them to the different zone images, and to the centrin images to check how much needs to be eroded

In [ ]:
# Show ventral and dorsal masks and the centrin_ventral_image, centrin_dorsal_image, polyedra_ventral_image, polyedra_dorsal_image
plt.imshow(ventral_mask, cmap='gray')
plt.title('Ventral Mask')
plt.show()
plt.imshow(dorsal_mask, cmap='gray')
plt.title('Dorsal Mask')
plt.show()
plt.imshow(centrin_ventral_image, cmap='gray')
plt.title('Centrin Ventral Image')
plt.show()
plt.imshow(centrin_dorsal_image, cmap='gray')
plt.title('Centrin Dorsal Image')
plt.show()
plt.imshow(polye_ventral_image, cmap='gray')
plt.title('Poly-e Ventral Image')
plt.show()
plt.imshow(polye_dorsal_image, cmap='gray')
plt.title('Poly-e Dorsal Image')
plt.show()

In [ ]:
# apply erosion and show the eroded images
iterations = 50  # Set the number of iterations as needed
eroded_ventral_mask = ventral_mask.copy()
eroded_dorsal_mask = dorsal_mask.copy()

for _ in range(iterations):
    eroded_ventral_mask = morphology.erosion(eroded_ventral_mask, morphology.disk(1))
    eroded_dorsal_mask = morphology.erosion(eroded_dorsal_mask, morphology.disk(1))

plt.imshow(eroded_ventral_mask, cmap='gray')
plt.title('Eroded Ventral Mask')
plt.show()
plt.imshow(eroded_dorsal_mask, cmap='gray')
plt.title('Eroded Dorsal Mask')
plt.show()

eroded_centrin_ventral_image = centrin_ventral_image * eroded_ventral_mask
eroded_centrin_dorsal_image = centrin_dorsal_image * eroded_dorsal_mask
eroded_polye_ventral_image = polye_ventral_image * eroded_ventral_mask
eroded_polye_dorsal_image = polye_dorsal_image * eroded_dorsal_mask

plt.imshow(eroded_centrin_ventral_image, cmap='gray')
plt.title('Eroded Centrin Ventral Image')
plt.show()
plt.imshow(eroded_centrin_dorsal_image, cmap='gray')
plt.title('Eroded Centrin Dorsal Image')
plt.show()
plt.imshow(eroded_polye_ventral_image, cmap='gray')
plt.title('Eroded Poly-e Ventral Image')
plt.show()
plt.imshow(eroded_polye_dorsal_image, cmap='gray')
plt.title('Eroded Poly-e Dorsal Image')
plt.show()

In [ ]:
# apply eroded masks to the images of the zones
# Apply the masks to the images to extract the zones
# # ZONE 1
# eroded_zone1_ventral_image_polye = zone1_mask * eroded_polye_ventral_image
# eroded_zone1_ventral_image_centrin = zone1_mask * eroded_centrin_ventral_image
# ZONE 1 smoothed
eroded_zone1_ventral_image_polye = polygon_mask_zone1 * (eroded_polye_ventral_image)
eroded_zone1_ventral_image_centrin = polygon_mask_zone1 * (eroded_centrin_ventral_image)

# # OA
# eroded_OA_ventral_image_polye = OA_mask * (255 - eroded_polye_ventral_image)
# eroded_OA_ventral_image_centrin = OA_mask * (255 - eroded_centrin_ventral_image)
# OA smoothed
eroded_OA_ventral_image_polye = polygon_mask_OA * (eroded_polye_ventral_image)
eroded_OA_ventral_image_centrin = polygon_mask_OA * (eroded_centrin_ventral_image)

# # ZONE 2 ventral
# eroded_zone2_ventral_image_polye = zone2_mask * (255 - eroded_polye_ventral_image)
# eroded_zone2_ventral_image_centrin = zone2_mask * (255 - eroded_centrin_ventral_image)
# ZONE 2 ventral smoothed
eroded_zone2_ventral_image_polye = zone2_mask_smoothed * eroded_polye_ventral_image
eroded_zone2_ventral_image_centrin = zone2_mask_smoothed * eroded_centrin_ventral_image
# ZONE 2 dorsal
eroded_zone2_dorsal_image_polye = dorsal_mask * (eroded_polye_dorsal_image)
eroded_zone2_dorsal_image_centrin = dorsal_mask * (eroded_centrin_dorsal_image)

# show the images
plt.imshow(eroded_zone1_ventral_image_polye, cmap='gray')
plt.title('Zone 1 Poly-e')
plt.show()
plt.imshow(eroded_zone1_ventral_image_centrin, cmap='gray')
plt.title('Zone 1 Centrin')
plt.show()
plt.imshow(eroded_OA_ventral_image_polye, cmap='gray')
plt.title('OA Poly-e')
plt.show()
plt.imshow(eroded_OA_ventral_image_centrin, cmap='gray')
plt.title('OA Centrin')
plt.show()
plt.imshow(eroded_zone2_ventral_image_polye, cmap='gray')
plt.title('Zone 2 Poly-e ventral')
plt.show()
plt.imshow(eroded_zone2_ventral_image_centrin, cmap='gray')
plt.title('Zone 2 Centrin ventral')
plt.show()
plt.imshow(eroded_zone2_dorsal_image_polye, cmap='gray')
plt.title('Zone 2 Poly-e dorsal')
plt.show()
plt.imshow(eroded_zone2_dorsal_image_centrin, cmap='gray')
plt.title('Zone 2 Centrin dorsal')
plt.show()

# Save the images
output_path = image_path[:-4] + '_zone1_ventral_polye_eroded.tif'
io.imsave(output_path, eroded_zone1_ventral_image_polye.astype(np.uint8))

output_path = image_path2[:-4] + '_zone1_ventral_centrin_eroded.tif'
io.imsave(output_path, eroded_zone1_ventral_image_centrin.astype(np.uint8))

output_path = image_path[:-4] + '_OA_ventral_polye_eroded.tif'
io.imsave(output_path, eroded_OA_ventral_image_polye.astype(np.uint8))

output_path = image_path2[:-4] + '_OA_ventral_centrin_eroded.tif'
io.imsave(output_path, eroded_OA_ventral_image_centrin.astype(np.uint8))

output_path = image_path[:-4] + '_zone2_ventral_polye_eroded.tif'
io.imsave(output_path, eroded_zone2_ventral_image_polye.astype(np.uint8))

output_path = image_path2[:-4] + '_zone2_ventral_centrin_eroded.tif'
io.imsave(output_path, eroded_zone2_ventral_image_centrin.astype(np.uint8))

output_path = image_path4[:-4] + '_zone2_dorsal_polye_eroded.tif'
io.imsave(output_path, eroded_zone2_dorsal_image_polye.astype(np.uint8))

output_path = image_path3[:-4] + '_zone2_dorsal_centrin_eroded.tif'
io.imsave(output_path, eroded_zone2_dorsal_image_centrin.astype(np.uint8))
